In [ ]:
import torch
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from facenet_pytorch import InceptionResnetV1

# Constants
IMAGE_SIZE = 160  # Adjusted size for InceptionResnetV1
BATCH_SIZE = 32
CHANNELS = 3

# Function to read and preprocess images
def read_and_preprocess(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = np.array(img, dtype='float32') 
    img = img / 255 
    return img

# Path to the dataset
image_path = 'C:/Users/roshi/PycharmProjects/DEEPTRUTH/Model Creation/data/'

# Lists to store images and labels
X = [] 
y = [] 

# Labels: 0 - Real, 1 - Fake
for folder in os.scandir(image_path):
    for entry in os.scandir(image_path + folder.name):
        X.append(read_and_preprocess(image_path + folder.name + '/' + entry.name))
        y.append(0 if folder.name[0] == 'r' else 1)

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Train-Test-Validation Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=123)
X_test, X_val, y_test, y_val = train_test_split(X_val, y_val, test_size=0.5, shuffle=True, stratify=y_val, random_state=123)

# InceptionResnetV1 Model
base_model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=1, device='cuda' if torch.cuda.is_available() else 'cpu')

# Customize the model head
x = base_model.logits.in_features
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=2, device='cuda' if torch.cuda.is_available() else 'cpu')

model.logits = nn.Linear(x, 2)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

# Callbacks for early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
checkpointer = ModelCheckpoint(filepath="inceptionresnetv1_weights.h5", verbose=1, save_best_only=True)

# Model training
history = model.fit(X_train, y_train, epochs=32, validation_data=(X_val, y_val),
                    batch_size=BATCH_SIZE, shuffle=True, callbacks=[early_stopping, checkpointer])

# Save the PyTorch model as a checkpoint
torch_model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=1, device='cuda' if torch.cuda.is_available() else 'cpu')
torch_model.logits = nn.Linear(x, 2)
torch_model.load_state_dict(model.state_dict())

# Save the model checkpoint
torch.save(torch_model.state_dict(), "resnetinceptionv1_epoch_32.pth")
